# DS542 Midterm Part-3

### Transfer Learning – DenseNet121 (Pretrained)

#### 10 Epochs

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "DenseNet121_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 10,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model: Pretrained DenseNet121
    # -----------------------------
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.classifier.in_features, 100)
    )
    model = model.to(CONFIG["device"])

    # -----------------------------
    # Loss, Optimizer, Scheduler
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B Setup
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="DenseNet121 Ep10")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_densenet121_ep10.pth")
            wandb.save("best_model_part3_densenet121_ep10.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_densenet121_ep10.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3densenet_ep10.csv", index=False)
    print("submission_ood_part3densenet_ep10.csv created successfully.")

if __name__ == '__main__':
    main()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: samritha (samritha-boston-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/10 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 2/10 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3densenet_ep10.csv created successfully.


#### 20 Epochs (continued from pretrained DenseNet121 at Epoch 10)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+11}/20 [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "DenseNet121_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 10,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model & Load Checkpoint
    # -----------------------------
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.classifier.in_features, 100)
    )
    model = model.to(CONFIG["device"])

    checkpoint_path = "best_model_part3_densenet121_ep10.pth"
    model.load_state_dict(torch.load(checkpoint_path, map_location=CONFIG["device"]))
    print(f"Loaded model weights from: {checkpoint_path}")

    # -----------------------------
    # Re-Init Optimizer & Scheduler for Fine-tune Phase 2
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B New Fine-Tuning Run
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="DenseNet121 Ep20")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 11,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_densenet121_ep20.pth")
            wandb.save("best_model_part3_densenet121_ep20.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_densenet121_ep20.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3densenet_ep20.csv", index=False)
    print("submission_ood_part3densenet_ep20.csv created successfully.")

if __name__ == '__main__':
    main()


Loaded model weights from: best_model_part3_densenet121_ep10.pth


Epoch 11/20 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 12/20 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3densenet_ep20.csv created successfully.


#### 30 Epochs (continued from pretrained DenseNet121 at Epoch 20)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+21}/30 [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "DenseNet121_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 10,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model & Load Checkpoint
    # -----------------------------
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.classifier.in_features, 100)
    )
    model = model.to(CONFIG["device"])

    checkpoint_path = "best_model_part3_densenet121_ep20.pth"
    model.load_state_dict(torch.load(checkpoint_path, map_location=CONFIG["device"]))
    print(f"Loaded model weights from: {checkpoint_path}")

    # -----------------------------
    # Re-Init Optimizer & Scheduler for Fine-tune Phase 3
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B New Fine-Tuning Run
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="DenseNet121 Ep30")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 21,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_densenet121_ep30.pth")
            wandb.save("best_model_part3_densenet121_ep30.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_densenet121_ep30.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3densenet_ep30.csv", index=False)
    print("submission_ood_part3densenet_ep30.csv created successfully.")

if __name__ == '__main__':
    main()


Loaded model weights from: best_model_part3_densenet121_ep20.pth


Epoch 21/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 22/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3densenet_ep30.csv created successfully.


#### 40 Epochs (continued from pretrained DenseNet121 at Epoch 30)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+31}/40 [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "DenseNet121_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 10,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model & Load Checkpoint
    # -----------------------------
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.classifier.in_features, 100)
    )
    model = model.to(CONFIG["device"])

    checkpoint_path = "best_model_part3_densenet121_ep30.pth"
    model.load_state_dict(torch.load(checkpoint_path, map_location=CONFIG["device"]))
    print(f"Loaded model weights from: {checkpoint_path}")

    # -----------------------------
    # Re-Init Optimizer & Scheduler for Fine-tune Phase 4
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B New Fine-Tuning Run
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="DenseNet121 Ep40",)
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 31,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_densenet121_ep40.pth")
            wandb.save("best_model_part3_densenet121_ep40.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_densenet121_ep40.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3densenet_ep40.csv", index=False)
    print("submission_ood_part3densenet_ep40.csv created successfully.")

if __name__ == '__main__':
    main()


Loaded model weights from: best_model_part3_densenet121_ep30.pth


Epoch 31/40 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 32/40 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3densenet_ep40.csv created successfully.


#### 50 Epochs (continued from pretrained DenseNet121 at Epoch 40)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+41}/50 [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "DenseNet121_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 10,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model & Load Checkpoint
    # -----------------------------
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.classifier.in_features, 100)
    )
    model = model.to(CONFIG["device"])

    checkpoint_path = "best_model_part3_densenet121_ep40.pth"
    model.load_state_dict(torch.load(checkpoint_path, map_location=CONFIG["device"]))
    print(f"Loaded model weights from: {checkpoint_path}")

    # -----------------------------
    # Re-Init Optimizer & Scheduler for Fine-tune Phase 5
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B New Fine-Tuning Run
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="DenseNet121 Ep50")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 41,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_densenet121_ep50.pth")
            wandb.save("best_model_part3_densenet121_ep50.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_densenet121_ep50.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3densenet_ep50.csv", index=False)
    print("submission_ood_part3densenet_ep50.csv created successfully.")

if __name__ == '__main__':
    main()


Loaded model weights from: best_model_part3_densenet121_ep40.pth


Epoch 41/50 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 42/50 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3densenet_ep50.csv created successfully.


### Transfer Learning – ConvNeXt_Tiny (Pretrained)

#### 30 Epochs

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil
import os

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# MC Dropout Prediction
# -----------------------------
def predict_with_mc_dropout(model, dataloader, CONFIG, T=1.5, num_passes=5):
    model.train()  # Enable dropout during inference
    all_probs = []

    with torch.no_grad():
        for _ in range(num_passes):
            probs = []
            for inputs, _ in dataloader:
                inputs = inputs.to(CONFIG["device"])
                outputs = model(inputs) / T
                probs.append(torch.softmax(outputs, dim=1).cpu())
            all_probs.append(torch.cat(probs, dim=0))

    mean_probs = torch.stack(all_probs).mean(0)
    return mean_probs

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "ConvNeXt_Tiny_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 0.001,
        "epochs": 30,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
        "temperature": 1.5,
        "mc_passes": 5,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------    
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model & Load Checkpoint
    # -----------------------------
    model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
    model.classifier[2] = nn.Linear(model.classifier[2].in_features, 100)
    model = model.to(CONFIG["device"])

    # -----------------------------
    # Loss, Optimizer, Scheduler
    # -----------------------------
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B Setup
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="ConvNeXtTiny Ep30")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_convnext_ep30.pth")
            wandb.save("best_model_part3_convnext_ep30.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_convnext_ep30.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy (MC Dropout + T={CONFIG['temperature']}): {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3convnext_ep30.csv", index=False)
    print("submission_ood_part3convnext_ep30.csv created successfully.")

if __name__ == '__main__':
    main()


Epoch 1/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 2/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 3/30 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3convnext_ep30.csv created successfully.


#### 50 Epochs (continued from pretrained ConvNeXt_Tiny at Epoch 30)

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm.auto import tqdm
import wandb
import numpy as np
import random
import shutil
import os

# -----------------------------
# Reproducibility
# -----------------------------
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# -----------------------------
# MC Dropout Prediction
# -----------------------------
def predict_with_mc_dropout(model, dataloader, CONFIG, T=1.5, num_passes=5):
    model.train()  # Enable dropout during inference
    all_probs = []
    with torch.no_grad():
        for _ in range(num_passes):
            probs = []
            for inputs, _ in dataloader:
                inputs = inputs.to(CONFIG["device"])
                outputs = model(inputs) / T
                probs.append(torch.softmax(outputs, dim=1).cpu())
            all_probs.append(torch.cat(probs, dim=0))
    return torch.stack(all_probs).mean(0)

# -----------------------------
# Training Function
# -----------------------------
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    device = CONFIG["device"]
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    progress = tqdm(trainloader, desc=f"Epoch {epoch+31}/50 [Train]", leave=False)

    for inputs, labels in progress:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress.set_postfix({
            "loss": running_loss / (total / CONFIG["batch_size"]),
            "acc": 100. * correct / total
        })

    return running_loss / len(trainloader), 100. * correct / total

# -----------------------------
# Validation Function
# -----------------------------
def validate(model, valloader, criterion, CONFIG):
    device = CONFIG["device"]
    model.eval()
    running_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        progress = tqdm(valloader, desc="[Validate]", leave=False)
        for inputs, labels in progress:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress.set_postfix({
                "loss": running_loss / (total / CONFIG["batch_size"]),
                "acc": 100. * correct / total
            })

    return running_loss / len(valloader), 100. * correct / total

# -----------------------------
# Main Function
# -----------------------------
def main():
    CONFIG = {
        "model": "ConvNeXt_Tiny_Pretrained_FT",
        "batch_size": 64,
        "learning_rate": 5e-5,
        "epochs": 20,
        "num_workers": 0,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
        "temperature": 1.5,
        "mc_passes": 5,
    }

    set_seed(CONFIG["seed"])

    # -----------------------------
    # Data Transforms (Augmentation)
    # -----------------------------
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2, 0.2, 0.2),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762)),
    ])

    # -----------------------------
    # Load CIFAR-100 Dataset
    # -----------------------------
    trainset_full = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=True, download=True, transform=transform_train)
    train_size = int(0.8 * len(trainset_full))
    val_size = len(trainset_full) - train_size
    trainset, valset = torch.utils.data.random_split(trainset_full, [train_size, val_size])

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=CONFIG["num_workers"])
    valloader = torch.utils.data.DataLoader(valset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])
    testset = torchvision.datasets.CIFAR100(root=CONFIG["data_dir"], train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=CONFIG["num_workers"])

    # -----------------------------
    # Define Model & Load Checkpoint
    # -----------------------------
    model = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
    model.classifier[2] = nn.Linear(model.classifier[2].in_features, 100)
    model = model.to(CONFIG["device"])
    
    checkpoint_path = "best_model_part3_convnext_ep30.pth"
    model.load_state_dict(torch.load(checkpoint_path, map_location=CONFIG["device"]))
    print(f"Loaded model weights from: {checkpoint_path}")


    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

    # -----------------------------
    # W&B Setup
    # -----------------------------
    wandb.init(project=CONFIG["wandb_project"], config=CONFIG, name="ConvNeXtTiny Ep50")
    wandb.watch(model)

    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 31,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model_part3_convnext_ep50.pth")
            wandb.save("best_model_part3_convnext_ep50.pth")

    wandb.finish()

    # -----------------------------
    # Evaluation & Submission
    # -----------------------------
    import eval_cifar100
    import eval_ood

    shutil.copy("best_model_part3_convnext_ep50.pth", "best_model.pth")

    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy (MC Dropout + T={CONFIG['temperature']}): {clean_accuracy:.2f}%")

    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3convnext_ep50.csv", index=False)
    print("submission_ood_part3convnext_ep50.csv created successfully.")

if __name__ == '__main__':
    main()


Loaded model weights from: best_model_part3_convnext_ep30.pth


Epoch 31/50 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 32/50 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/157 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3convnext_ep50.csv created successfully.
